### Fase 1: La Plaza 
Te encuentras en la plaza de la ciudad, donde los habitantes desarrollan sus diversas profesiones: panaderos, herreros, guardias, escuderos...   
En el siguiente enlace https://cutt.ly/peVAXig  encontrarás un archivo que contiene un número por cada persona que se encuentra en la plaza. Suma los números que correspondan a los guardias y podrás pasar a la siguiente fase.


### Fase 2: La Biblioteca 
Te encuentras en la biblioteca. La cantidad de libros es inmensa. Por suerte, en la plaza conseguiste lo que necesitas. Encuentra el libro que esté en la posición del número que has guardado y accede a su contenido.   
Descarga el archivo books.xml creando una conexión al servidor FTP  
`Servidor: f31-preview.runhosting.com  
Usuario: 4009006_DATOS   
Contraseña: Rafa9999 `

### Fase 3: El Libro 
¡Rápido! El tiempo se agota. Para pasar a la siguiente frase deberás encontrar la palabra de seis letras que más se repite en el libro. ¡Usa tu magia!

### Fase 4: La Tienda   
Tu siguiente destino es la tienda de comestibles cercana a la Iglesia. Solo te dejarán entrar si conoces la contraseña, la cual conseguiste en la fase anterior. Una vez dentro, averigua cuál es el producto más vendido en la historia de la tienda, y cómpralo. 
Descarga el archivo del siguiente enlace: https://cutt.ly/peVALXr, descomprímelo utilizando la respuesta de la fase anterior como contraseña. Carga la base de datos en Python, y encuentra el nombre 
del producto más vendido.


### FASE 5: El Castillo   
¡Ya casi estás! Solo te falta entrar al castillo. Para ello, deberás sobornar al guardia elaborando la receta más calórica con el alimento que conseguiste en la fase anterior.